In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\DE Optimization\initFiles"
potential = 'DW'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    #tolerance.append(optimzer['tolerance'])
    #abs_tolerance.append(optimzer['abs_tolerance'])
    #strategy.append(optimzer['strategy'])
    #popsize.append(optimzer['popsize'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [3]:
df = pd.DataFrame({
    'folder': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    #'tolerance': tolerance,
    #'abs_tolerance': abs_tolerance,
    #'strategy': strategy,
    #'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [4]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
0,custom,40,1024,40/40,10000,132,7.31637e-14,4.15834e-05,8.91599e-01,0:09:25.442648
3,random,40,1024,40/40,10000,164,7.31637e-14,4.13804e-05,8.91599e-01,1:21:02.622138
2,latinhypercube,40,1024,40/40,10000,169,7.31637e-14,4.13398e-05,8.91599e-01,1:22:53.386984
1,halton,40,1024,40/40,10000,168,7.31637e-14,4.12992e-05,8.91599e-01,1:21:18.502206
4,sobol,40,1024,40/40,10000,164,7.31637e-14,4.12586e-05,8.91599e-01,2:16:55.622076


In [6]:
from tabulate import tabulate
df = df.drop(columns=['num_vqe','shots','maxiter']).sort_values('Mean Iter',ascending=True)

latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{llrrrrl}
\hline
 folder         & Converged Runs   &   Mean Iter &   Delta min E &   Delta median E &    Exact & Time           \\
\hline
 custom         & 40/40            &         132 &   7.31637e-14 &      4.15834e-05 & 0.891599 & 0:09:25.442648 \\
 random         & 40/40            &         164 &   7.31637e-14 &      4.13804e-05 & 0.891599 & 1:21:02.622138 \\
 sobol          & 40/40            &         164 &   7.31637e-14 &      4.12586e-05 & 0.891599 & 2:16:55.622076 \\
 halton         & 40/40            &         168 &   7.31637e-14 &      4.12992e-05 & 0.891599 & 1:21:18.502206 \\
 latinhypercube & 40/40            &         169 &   7.31637e-14 &      4.13398e-05 & 0.891599 & 1:22:53.386984 \\
\hline
\end{tabular}
